# Demo: Intelligent Prerequisite Detection with PBMC3k Data

This notebook demonstrates the **hybrid prerequisite detection system** using real PBMC3k single-cell data.

## System Architecture

The system uses **two layers** to intelligently handle function prerequisites:

- **Layer 1: Runtime Data Inspection** - Examines actual data state (no hardcoding)
- **Layer 2: LLM Inference** - Analyzes function documentation and reasons about prerequisites

## What You'll See

1. **Data State Inspection** - How the system examines your data
2. **Smart Classification** - How it decides simple vs. complex tasks
3. **Auto-Fix Prerequisites** - Automatically running simple prerequisites
4. **Workflow Escalation** - Detecting when full pipeline is needed
5. **Real Agent Usage** - Using ov.Agent with intelligent prerequisite handling

---

## Setup

In [ ]:
import omicverse as ov
import scanpy as sc
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Set plot style
ov.plot_set()

print(f"OmicVerse version: {ov.__version__}")
print("✅ Setup complete")

## Load PBMC3k Data

We'll use the standard PBMC3k dataset from scanpy.

In [ ]:
# Load PBMC3k data
adata = sc.datasets.pbmc3k()

print(f"📊 Loaded PBMC3k data: {adata.shape[0]} cells × {adata.shape[1]} genes")
print(f"Data type: {type(adata)}")
adata

---

# Part 1: Layer 1 - Runtime Data Inspection

Let's see how **Layer 1** examines the current data state.

In [ ]:
from omicverse.utils.smart_agent import DataStateInspector

# Inspect raw data state
print("🔍 Layer 1: Inspecting RAW DATA state...\n")

state_raw = DataStateInspector.inspect(adata)

print("📋 Data State Report:")
print(f"  Shape: {state_raw['shape'][0]:,} cells × {state_raw['shape'][1]:,} genes")
print(f"  Available layers: {state_raw['available']['layers']}")
print(f"  Available obsm: {state_raw['available']['obsm']}")
print(f"  Available uns: {state_raw['available']['uns'][:5]}...")
print(f"  Detected capabilities: {state_raw['capabilities']}")

print("\n💡 Interpretation:")
if not state_raw['capabilities']:
    print("  ⚠️  This is RAW data with no preprocessing")
    print("  ⚠️  Functions requiring preprocessing will need full pipeline")

### Human-Readable Summary

Layer 1 can generate a human-readable summary for LLM prompts:

In [ ]:
summary = DataStateInspector.get_readable_summary(adata)
print(summary)

### Function Compatibility Check

Layer 1 can check if a function is compatible with current data:

In [ ]:
# Check if PCA can run on raw data
print("🔍 Checking PCA compatibility on raw data...\n")

compat = DataStateInspector.check_compatibility(
    adata,
    function_name='pca',
    function_signature="(adata, layer='scaled', n_pcs=50)",
    function_category='preprocessing'
)

print(f"Likely compatible: {compat['likely_compatible']}")
print(f"\nWarnings:")
for warning in compat['warnings']:
    print(f"  ⚠️  {warning}")

print(f"\nSuggestions:")
for suggestion in compat['suggestions']:
    print(f"  💡 {suggestion}")

print(f"\nReasoning: {compat['reasoning']}")

---

# Part 2: Data State-Aware Classification

The system uses data state to classify tasks as **SIMPLE** or **COMPLEX**.

In [ ]:
import asyncio
import os

# Check if API key is available
has_api_key = bool(os.getenv('OPENAI_API_KEY') or os.getenv('ANTHROPIC_API_KEY'))

if not has_api_key:
    print("⚠️  No API key found. Skipping agent examples.")
    print("   Set OPENAI_API_KEY or ANTHROPIC_API_KEY to test agent functionality.\n")
else:
    from omicverse.utils.smart_agent import OmicVerseAgent
    
    # Initialize agent
    agent = OmicVerseAgent(model='gpt-4o-mini')
    
    print("\n🤖 Testing Task Classification with Data State Awareness...\n")
    
    # Test 1: PCA on raw data (should be COMPLEX)
    async def test_classification():
        print("Test 1: PCA on RAW DATA")
        complexity = await agent._analyze_task_complexity("Run PCA", adata)
        print(f"  Request: 'Run PCA'")
        print(f"  Data state: Raw (no preprocessing)")
        print(f"  Classification: {complexity.upper()}")
        print(f"  Reason: PCA needs preprocessing, but data is raw\n")
        
        return complexity
    
    # Run async test
    complexity_result = await test_classification()
    
    if complexity_result == 'complex':
        print("✅ CORRECT: System detected this needs full preprocessing pipeline")
    else:
        print("⚠️  Unexpected classification")

---

# Part 3: Preprocessing the Data

Let's preprocess the data step by step and observe how the system's assessment changes.

## Step 1: Quality Control

In [ ]:
print("🔬 Running Quality Control...\n")

# QC with standard thresholds
adata_qc = adata.copy()
adata_qc = ov.pp.qc(
    adata_qc,
    tresh={'mito_perc': 0.2, 'nUMIs': 500, 'detected_genes': 250}
)

print(f"✅ QC complete: {adata_qc.shape[0]} cells remaining")

# Check data state after QC
state_qc = DataStateInspector.inspect(adata_qc)
print(f"\n📋 Data State After QC:")
print(f"  Capabilities: {state_qc['capabilities']}")
print(f"  Layers: {state_qc['available']['layers']}")

## Step 2: Preprocessing (Normalization + Scaling)

In [ ]:
print("🔬 Running Preprocessing (normalization + HVG + scaling)...\n")

# Store raw counts
adata_qc.layers['counts'] = adata_qc.X.copy()

# Preprocess
adata_preprocessed = adata_qc.copy()
adata_preprocessed = ov.pp.preprocess(
    adata_preprocessed,
    mode='shiftlog|pearson',
    n_HVGs=2000,
    target_sum=1e4
)

# Scale
adata_preprocessed = ov.pp.scale(adata_preprocessed)

print(f"✅ Preprocessing complete")

# Check data state after preprocessing
state_preprocessed = DataStateInspector.inspect(adata_preprocessed)
print(f"\n📋 Data State After Preprocessing:")
print(f"  Capabilities: {state_preprocessed['capabilities']}")
print(f"  Layers: {state_preprocessed['available']['layers']}")

print("\n💡 Now the data is ready for PCA!")

### Re-check PCA Compatibility

In [ ]:
print("🔍 Re-checking PCA compatibility on PREPROCESSED data...\n")

compat_preprocessed = DataStateInspector.check_compatibility(
    adata_preprocessed,
    function_name='pca',
    function_signature="(adata, layer='scaled', n_pcs=50)",
    function_category='preprocessing'
)

print(f"Likely compatible: {compat_preprocessed['likely_compatible']}")
print(f"Warnings: {len(compat_preprocessed['warnings'])} (was {len(compat['warnings'])} before)")
print(f"\nReasoning: {compat_preprocessed['reasoning']}")

if len(compat_preprocessed['warnings']) == 0:
    print("\n✅ SUCCESS: Data is now ready for PCA!")

### Test Classification on Preprocessed Data

In [ ]:
if has_api_key:
    print("🤖 Testing Classification on PREPROCESSED DATA...\n")
    
    async def test_preprocessed_classification():
        complexity = await agent._analyze_task_complexity("Run PCA", adata_preprocessed)
        print(f"  Request: 'Run PCA'")
        print(f"  Data state: Preprocessed (has 'scaled' layer)")
        print(f"  Classification: {complexity.upper()}")
        print(f"  Reason: Data is ready, can execute directly\n")
        return complexity
    
    complexity_preprocessed = await test_preprocessed_classification()
    
    if complexity_preprocessed == 'simple':
        print("✅ CORRECT: System detected data is ready for direct execution")
        print("\n💡 Compare:")
        print(f"  - Raw data → COMPLEX (needs full pipeline)")
        print(f"  - Preprocessed data → SIMPLE (ready to execute)")
        print("\n🎉 Data state awareness working perfectly!")

---

# Part 4: Layer 2 - LLM-Based Prerequisite Inference

Now let's see **Layer 2** in action - LLM reasoning about prerequisites.

In [ ]:
if has_api_key:
    from omicverse.utils.smart_agent import LLMPrerequisiteInference
    from omicverse.utils.registry import _global_registry
    
    print("🧠 Layer 2: LLM-Based Prerequisite Inference\n")
    
    # Get PCA function info from registry
    pca_results = _global_registry.find('pca')
    if pca_results:
        pca_info = pca_results[0]
        
        # Test on raw data
        print("Test 1: PCA on RAW DATA\n")
        
        async def test_llm_inference_raw():
            inference_engine = agent._prerequisite_inference
            
            result = await inference_engine.infer_prerequisites(
                function_name='pca',
                function_info=pca_info,
                data_state=state_raw,
                skill_context=None
            )
            
            print(f"  Can run: {result['can_run']}")
            print(f"  Confidence: {result['confidence']:.0%}")
            print(f"  Complexity: {result['complexity'].upper()}")
            print(f"  Missing items: {', '.join(result['missing_items'])}")
            print(f"  Required steps: {' → '.join(result['required_steps'])}")
            print(f"  Auto-fixable: {result['auto_fixable']}")
            print(f"\n  LLM Reasoning: {result['reasoning']}\n")
            
            return result
        
        result_raw = await test_llm_inference_raw()
        
        # Test on preprocessed data
        print("\nTest 2: PCA on PREPROCESSED DATA\n")
        
        async def test_llm_inference_preprocessed():
            inference_engine = agent._prerequisite_inference
            
            result = await inference_engine.infer_prerequisites(
                function_name='pca',
                function_info=pca_info,
                data_state=state_preprocessed,
                skill_context=None
            )
            
            print(f"  Can run: {result['can_run']}")
            print(f"  Confidence: {result['confidence']:.0%}")
            print(f"  Complexity: {result['complexity'].upper()}")
            print(f"  Missing items: {', '.join(result['missing_items']) if result['missing_items'] else 'None'}")
            print(f"\n  LLM Reasoning: {result['reasoning']}\n")
            
            return result
        
        result_preprocessed = await test_llm_inference_preprocessed()
        
        print("\n✅ Layer 2 demonstrates intelligent reasoning:")
        print(f"  - Analyzes function documentation")
        print(f"  - Compares with current data state")
        print(f"  - Provides structured recommendations")
        print(f"  - No hardcoding - learns from documentation!")

---

# Part 5: Complete Analysis Pipeline

Let's run a complete analysis to demonstrate the system in a realistic workflow.

## Run PCA (Now that data is preprocessed)

In [ ]:
print("🔬 Running PCA on preprocessed data...\n")

adata_pca = adata_preprocessed.copy()
adata_pca = ov.pp.pca(adata_pca, layer='scaled', n_pcs=50)

print(f"✅ PCA complete: {adata_pca.obsm['X_pca'].shape}")

# Check new state
state_pca = DataStateInspector.inspect(adata_pca)
print(f"\n📋 Data State After PCA:")
print(f"  Capabilities: {state_pca['capabilities']}")
print(f"  Available obsm: {state_pca['available']['obsm']}")

## Test: Leiden Clustering (Missing Neighbors)

This demonstrates **auto-fixing** - clustering needs neighbors, but the system should detect it can auto-run neighbors since we have PCA.

In [ ]:
if has_api_key:
    print("🧪 Testing: Leiden Clustering (data has PCA, missing neighbors)\n")
    
    # Get leiden function info
    leiden_results = _global_registry.find('leiden')
    if leiden_results:
        leiden_info = leiden_results[0]
        
        async def test_leiden_inference():
            inference_engine = agent._prerequisite_inference
            
            result = await inference_engine.infer_prerequisites(
                function_name='leiden',
                function_info=leiden_info,
                data_state=state_pca,
                skill_context=None
            )
            
            print(f"  Can run: {result['can_run']}")
            print(f"  Confidence: {result['confidence']:.0%}")
            print(f"  Complexity: {result['complexity'].upper()}")
            print(f"  Missing items: {', '.join(result['missing_items'])}")
            print(f"  Required steps: {' → '.join(result['required_steps'])}")
            print(f"  Auto-fixable: {result['auto_fixable']}")
            print(f"\n  LLM Reasoning: {result['reasoning']}\n")
            
            return result
        
        leiden_result = await test_leiden_inference()
        
        if leiden_result['auto_fixable']:
            print("\n✅ EXCELLENT: Layer 2 detected this is AUTO-FIXABLE")
            print("   The agent can automatically run neighbors before leiden")
            print("   This is a 1-step fix, no need for full workflow!")

## Actually Run the Auto-Fix

Let's manually demonstrate what the agent would do automatically:

In [ ]:
print("🔧 Demonstrating Auto-Fix: Neighbors + Leiden\n")

adata_clustered = adata_pca.copy()

# Auto-fix: Run neighbors first
print("  Step 1: Auto-running neighbors (prerequisite)...")
adata_clustered = ov.pp.neighbors(
    adata_clustered,
    n_neighbors=15,
    use_rep='X_pca'
)
print("  ✅ Neighbors computed")

# Now run leiden
print("  Step 2: Running leiden clustering...")
adata_clustered = ov.pp.leiden(adata_clustered, resolution=1.0)
print(f"  ✅ Leiden complete: {adata_clustered.obs['leiden'].nunique()} clusters")

# Check final state
state_final = DataStateInspector.inspect(adata_clustered)
print(f"\n📋 Final Data State:")
print(f"  Capabilities: {state_final['capabilities']}")
print(f"  Available uns: neighbors = {'neighbors' in state_final['available']['uns']}")
print(f"  Clustering columns: {state_final.get('clustering_columns', [])}")

## Visualize Results

In [ ]:
# Compute UMAP for visualization
print("🎨 Computing UMAP for visualization...\n")
adata_viz = adata_clustered.copy()
adata_viz = ov.pp.umap(adata_viz)

# Plot
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot by leiden clusters
sc.pl.umap(adata_viz, color='leiden', ax=axes[0], show=False, title='Leiden Clusters')

# Plot by total counts
sc.pl.umap(adata_viz, color='n_genes', ax=axes[1], show=False, title='Number of Genes')

plt.tight_layout()
plt.show()

print(f"\n✅ Analysis complete!")
print(f"   {adata_viz.shape[0]} cells")
print(f"   {adata_viz.obs['leiden'].nunique()} clusters")
print(f"   Ready for downstream analysis")

---

# Part 6: Using the Agent (Optional)

If you have an API key configured, you can use the actual agent with intelligent prerequisite handling.

**Note**: This section requires `OPENAI_API_KEY` or `ANTHROPIC_API_KEY` environment variable.

In [ ]:
if has_api_key:
    print("🤖 Using OmicVerse Agent with Intelligent Prerequisite Handling\n")
    print("="*70)
    
    # Example 1: Agent on preprocessed data (should work directly)
    print("\nExample 1: PCA on preprocessed data")
    print("-" * 70)
    
    async def agent_example_1():
        adata_test = adata_preprocessed.copy()
        result = await agent.run("Run PCA with 30 components", adata_test)
        print(f"\n✅ Agent successfully executed PCA")
        print(f"   Result shape: {result.obsm['X_pca'].shape if 'X_pca' in result.obsm else 'N/A'}")
        return result
    
    result1 = await agent_example_1()
    
    print("\n" + "="*70)
    print("\n💡 What happened:")
    print("   1. Agent checked data state (Layer 1)")
    print("   2. Found 'scaled' layer available")
    print("   3. Classified as SIMPLE task")
    print("   4. Executed PCA directly")
    print("   5. No workflow escalation needed!")
else:
    print("⏭️  Skipping agent examples (no API key configured)")
    print("\nTo use the agent:")
    print("  1. Set OPENAI_API_KEY or ANTHROPIC_API_KEY")
    print("  2. Re-run this cell")
    print("\nThe agent will use the hybrid prerequisite detection system")
    print("to intelligently handle function prerequisites automatically.")

---

# Summary

## What We Demonstrated

### ✅ Layer 1: Runtime Data Inspection
- **Examines actual data state** (layers, obsm, uns, capabilities)
- **No hardcoding** - just reports facts
- **Compatibility checking** - warns about missing prerequisites
- **Human-readable summaries** for LLM context

### ✅ Layer 2: LLM-Based Prerequisite Inference
- **Analyzes function documentation** intelligently
- **Compares needs vs. current state**
- **Provides structured recommendations** (auto-fix vs. workflow)
- **Learns from documentation** - no hardcoding!
- **Caches results** for performance

### ✅ Data State-Aware Classification
- **PCA on raw data** → COMPLEX (needs full pipeline)
- **PCA on preprocessed data** → SIMPLE (ready to execute)
- **Clustering with PCA** → SIMPLE (can auto-run neighbors)
- **Clustering without PCA** → COMPLEX (needs preprocessing)

### ✅ Intelligent Auto-Fixing
- **1-step prerequisites** → Auto-run (e.g., neighbors before leiden)
- **Multi-step prerequisites** → Escalate to workflow
- **Defensive validation** → Checks before execution

## Key Benefits

| Feature | Old System | New Hybrid System |
|---------|-----------|-------------------|
| **Prerequisite checking** | ❌ None | ✅ Automatic (2 layers) |
| **Hardcoding** | ❌ Would need 100+ rules | ✅ Zero hardcoding |
| **Maintenance** | ❌ Manual updates | ✅ Self-maintaining |
| **Custom workflows** | ❌ Not supported | ✅ Fully supported |
| **Novel functions** | ❌ Needs new rules | ✅ Learns automatically |
| **Intelligence** | ❌ Pattern matching | ✅ LLM reasoning |

## Architecture Recap

```
Layer 1 (Runtime) → Layer 2 (LLM) → Classification → Smart Execution
     ↓                   ↓               ↓                ↓
  Facts about       Intelligent      SIMPLE vs.     Auto-fix or
  data state        reasoning        COMPLEX        Escalate
```

---

## Conclusion

The **hybrid prerequisite detection system** provides:

1. 🎯 **Accuracy** - Never executes functions on unprepared data
2. 🤖 **Intelligence** - Learns from documentation, not hardcoded rules
3. 🚀 **Performance** - Caching and smart classification
4. 🔮 **Future-proof** - Adapts to any workflow automatically
5. ✨ **User-friendly** - Auto-fixes simple issues transparently

**No hardcoding. No maintenance. Just intelligence.**

---